In [1]:
import pandas as pd

In [2]:
# Load and merge datasets based on foreign keys
train_raw = pd.read_csv('/content/drive/MyDrive/Energy Predictor/train.csv')
test_raw = pd.read_csv('/content/drive/MyDrive/Energy Predictor/test.csv')
building_meta = pd.read_csv('/content/drive/MyDrive/Energy Predictor/building_metadata.csv')
weather_train = pd.read_csv('/content/drive/MyDrive/Energy Predictor/weather_train.csv')
weather_test = pd.read_csv('/content/drive/MyDrive/Energy Predictor/weather_test.csv')

In [3]:
primary_use_categories = building_meta['primary_use'].unique()
p_u_code = []
for c in enumerate(primary_use_categories):
  p_u_code.append([c[0],c[1]])
p_u_df = pd.DataFrame(p_u_code, columns=['primary_use_code', 'primary_use'])

In [4]:
def convert_timestamp(df):
  df['year'] = pd.to_datetime(df['timestamp']).dt.year
  df['month'] = pd.to_datetime(df['timestamp']).dt.month
  df['day'] = pd.to_datetime(df['timestamp']).dt.day
  df['hour'] = pd.to_datetime(df['timestamp']).dt.hour

  return df

In [5]:
train_raw = convert_timestamp(train_raw)
train = pd.merge(train_raw, building_meta, on='building_id', how='left')
train = pd.merge(train, p_u_df, on='primary_use', how='left')
train = pd.merge(train, weather_train, on=['site_id','timestamp'], how='left')
train = train.drop(['building_id','timestamp','site_id','primary_use'], axis=1)

In [7]:
# import numpy as np

# df = train.replace(r'^\s*$', np.nan, regex=True)
# for col in df:
#     empty_rate = round(df[col].isna().mean()*100, 1)
#     print(col, empty_rate)

# The top 3 empty rate are: floor_count(82.7%), year_built(60.0%), and cloud_coverage(43.7%)
# Empty rate of other features were less than 20%
# Thus, these 3 features are droped

train = train.drop(['floor_count','year_built','cloud_coverage'], axis=1)

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 14 columns):
 #   Column              Dtype  
---  ------              -----  
 0   meter               int64  
 1   meter_reading       float64
 2   year                int64  
 3   month               int64  
 4   day                 int64  
 5   hour                int64  
 6   square_feet         int64  
 7   primary_use_code    int64  
 8   air_temperature     float64
 9   dew_temperature     float64
 10  precip_depth_1_hr   float64
 11  sea_level_pressure  float64
 12  wind_direction      float64
 13  wind_speed          float64
dtypes: float64(7), int64(7)
memory usage: 2.3 GB


In [8]:
# import seaborn as sns
# import pandas as pd
# from matplotlib import pyplot as plt

# train_new = pd.read_csv('/content/drive/MyDrive/Energy Predictor/train_new.csv')
# corr = train_new.corr()
# plt.figure(figsize=(20,10))
# sns.heatmap(corr, cmap="Greens",annot=True)
# plt.show()

# According to the correlation, 'year','month','day','hour','precip_depth_1_hr','wind_direction' were removed due to the low correclation
train = train.drop(['year','day', 'hour', 'precip_depth_1_hr', 'wind_direction'], axis=1)

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   meter               int64  
 1   meter_reading       float64
 2   month               int64  
 3   square_feet         int64  
 4   primary_use_code    int64  
 5   air_temperature     float64
 6   dew_temperature     float64
 7   sea_level_pressure  float64
 8   wind_speed          float64
dtypes: float64(5), int64(4)
memory usage: 1.5 GB


In [9]:
train.to_csv('/content/drive/MyDrive/Energy Predictor/train_new.csv', index=False)

In [10]:
test_raw = convert_timestamp(test_raw)
test = pd.merge(test_raw, building_meta, on='building_id', how='left')
test = pd.merge(test, p_u_df, on='primary_use', how='left')
test = pd.merge(test, weather_test, on=['site_id','timestamp'], how='left')
test = test.drop(['building_id','timestamp','site_id','primary_use'], axis=1)
test = test.drop(['floor_count','year_built','cloud_coverage'], axis=1)
test = test.drop(['year','day', 'hour', 'precip_depth_1_hr', 'wind_direction'], axis=1)
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41697600 entries, 0 to 41697599
Data columns (total 9 columns):
 #   Column              Dtype  
---  ------              -----  
 0   row_id              int64  
 1   meter               int64  
 2   month               int64  
 3   square_feet         int64  
 4   primary_use_code    int64  
 5   air_temperature     float64
 6   dew_temperature     float64
 7   sea_level_pressure  float64
 8   wind_speed          float64
dtypes: float64(4), int64(5)
memory usage: 3.1 GB


In [11]:
test.to_csv('/content/drive/MyDrive/Energy Predictor/test_new.csv', index=False)